## Imports

In [1]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models, datasets, transforms
from utils import train_model, evaluate_model
import random
import numpy as np

## Set seed for reproducibility

In [2]:
def set_seed(seed):
    """
    Set the seed for reproducibility.

    Args:
        seed (int): Seed value to set for random number generation.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed to be 42
set_seed(42)

## Resnet-18

In [3]:
# Use CUDA if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the resnet model to use
def create_resnet_model(num_classes):
    model = models.resnet18(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

## Objective Function

In [4]:
# Set the dataset folder
dataset_folder = '../data/lfw/'

In [5]:
def objective(trial):
    """
    Objective function for hyperparameter optimization using Optuna.

    This function is used by Optuna to suggest and evaluate hyperparameters for training
    a ResNet model. It performs the following steps:
    1. Suggests values for learning rate, batch size, and weight decay from predefined distributions.
    2. Loads and preprocesses the dataset.
    3. Creates data loaders for training, validation, and testing.
    4. Initializes a ResNet-18 model and sets up the optimizer and loss function.
    5. Trains the model using the suggested hyperparameters.
    6. Evaluates the model on the validation set and calculates accuracy.
    7. Returns the negative validation accuracy (as Optuna minimizes the objective function).

    Args:
        trial (optuna.Trial): An Optuna trial object used to suggest hyperparameter values.

    Returns:
        float: The negative validation accuracy of the model, which Optuna aims to minimize.
    """
    # Suggest hyperparameters
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)  
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64]) 
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2) 

    # Define transforms
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.43636918, 0.38563913, 0.34477144],
                             std=[0.29639485, 0.2698132, 0.26158142])
    ])

    # Load datasets with 8-1-1 split
    full_dataset = datasets.ImageFolder(root=dataset_folder, transform=transform)
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])
    val_size = int(0.5 * len(test_dataset))
    test_size = len(test_dataset) - val_size
    val_dataset, test_dataset = random_split(test_dataset, [val_size, test_size])

    # Create dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Create model
    num_classes = len(full_dataset.classes)
    model = create_resnet_model(num_classes)
    model.to(device)

    # Define optimizer and criterion
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Train model
    trained_model = train_model(model, train_dataloader, val_dataloader, num_epochs=50, lr=lr, patience=5)

    # Evaluate model
    val_accuracy = evaluate_model(trained_model, val_dataloader)[0]
    
    # Return the negative validation accuracy as Optuna tries to minimize the objective
    return 1 - val_accuracy

## Start the Bayesian Optimizer

In [6]:
study = optuna.create_study(direction='maximize')  # We are maximizing accuracy
study.optimize(objective, n_trials=50)  # Number of trials to perform

print('Best hyperparameters: ', study.best_params)
print('Best value: ', study.best_value)

[I 2024-09-14 17:37:01,759] A new study created in memory with name: no-name-e37d4323-5843-4866-93eb-4140cd9935b2
/tmp/ipykernel_1809568/2091348980.py:22: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/tmp/ipykernel_1809568/2091348980.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)


Epoch 1/50, Train Loss: 2.3551, Val Loss: 2.3452
Epoch 2/50, Train Loss: 2.2140, Val Loss: 2.3070
Epoch 3/50, Train Loss: 2.1138, Val Loss: 2.2440
Epoch 4/50, Train Loss: 2.0140, Val Loss: 2.1864
Epoch 5/50, Train Loss: 1.9186, Val Loss: 2.1535
Epoch 6/50, Train Loss: 1.8066, Val Loss: 2.1005
Epoch 7/50, Train Loss: 1.7036, Val Loss: 2.0454
Epoch 8/50, Train Loss: 1.5792, Val Loss: 2.0125
Epoch 9/50, Train Loss: 1.4416, Val Loss: 1.9414
Epoch 10/50, Train Loss: 1.3211, Val Loss: 1.8960
Epoch 11/50, Train Loss: 1.1920, Val Loss: 1.8289
Epoch 12/50, Train Loss: 1.0700, Val Loss: 1.8141
Epoch 13/50, Train Loss: 0.9423, Val Loss: 1.7763
Epoch 14/50, Train Loss: 0.8386, Val Loss: 1.7139
Epoch 15/50, Train Loss: 0.7456, Val Loss: 1.6675
Epoch 16/50, Train Loss: 0.6538, Val Loss: 1.6551
Epoch 17/50, Train Loss: 0.5780, Val Loss: 1.6260
Epoch 18/50, Train Loss: 0.5053, Val Loss: 1.6033
Epoch 19/50, Train Loss: 0.4516, Val Loss: 1.5919
Epoch 20/50, Train Loss: 0.4051, Val Loss: 1.5571
Epoch 21/

[I 2024-09-14 17:38:17,593] Trial 0 finished with value: 0.4383561611175537 and parameters: {'lr': 1.511617269436484e-05, 'batch_size': 64, 'weight_decay': 5.35352465955097e-06}. Best is trial 0 with value: 0.4383561611175537.


Accuracy: 0.5616
F1 Score: 0.5008
Epoch 1/50, Train Loss: 2.2628, Val Loss: 2.1193
Epoch 2/50, Train Loss: 1.7776, Val Loss: 1.8128
Epoch 3/50, Train Loss: 1.0666, Val Loss: 1.4353
Epoch 4/50, Train Loss: 0.4710, Val Loss: 1.2209
Epoch 5/50, Train Loss: 0.1855, Val Loss: 1.0550
Epoch 6/50, Train Loss: 0.0972, Val Loss: 0.8447
Epoch 7/50, Train Loss: 0.0547, Val Loss: 0.8068
Epoch 8/50, Train Loss: 0.0322, Val Loss: 0.7934
Epoch 9/50, Train Loss: 0.0282, Val Loss: 0.8695
Epoch 10/50, Train Loss: 0.0258, Val Loss: 0.8277
Epoch 11/50, Train Loss: 0.0174, Val Loss: 0.7471
Epoch 12/50, Train Loss: 0.0123, Val Loss: 0.7337
Epoch 13/50, Train Loss: 0.0097, Val Loss: 0.7511
Epoch 14/50, Train Loss: 0.0094, Val Loss: 0.7883
Epoch 15/50, Train Loss: 0.0127, Val Loss: 0.7524
Epoch 16/50, Train Loss: 0.0101, Val Loss: 0.8151
Epoch 17/50, Train Loss: 0.0088, Val Loss: 0.6969
Epoch 18/50, Train Loss: 0.0072, Val Loss: 0.6941
Epoch 19/50, Train Loss: 0.0064, Val Loss: 0.7117
Epoch 20/50, Train Loss: 

[I 2024-09-14 17:39:16,021] Trial 1 finished with value: 0.13013696670532227 and parameters: {'lr': 6.930427671141797e-05, 'batch_size': 16, 'weight_decay': 8.078068221910567e-05}. Best is trial 0 with value: 0.4383561611175537.


...

Epoch 24/50, Train Loss: 0.0250, Val Loss: 0.3299
Early stopping triggered after 24 epochs
Training complete
Accuracy: 0.9041
F1 Score: 0.8823
Epoch 1/50, Train Loss: 4.6326, Val Loss: 2.3055
Epoch 2/50, Train Loss: 2.2974, Val Loss: 2.2925
Epoch 3/50, Train Loss: 2.2890, Val Loss: 2.3201
Epoch 4/50, Train Loss: 2.3034, Val Loss: 2.3003
Epoch 5/50, Train Loss: 2.2795, Val Loss: 2.2856
Epoch 6/50, Train Loss: 2.2928, Val Loss: 2.3148
Epoch 7/50, Train Loss: 2.2580, Val Loss: 4.2917
Epoch 8/50, Train Loss: 2.2698, Val Loss: 2.2706
Epoch 9/50, Train Loss: 2.2236, Val Loss: 2.3984
Epoch 10/50, Train Loss: 2.1736, Val Loss: 2.2666
Epoch 11/50, Train Loss: 2.1162, Val Loss: 2.2737
Epoch 12/50, Train Loss: 1.9696, Val Loss: 2.5280
Epoch 13/50, Train Loss: 1.9180, Val Loss: 2.2840
Epoch 14/50, Train Loss: 1.8639, Val Loss: 3.1497
Epoch 15/50, Train Loss: 1.8458, Val Loss: 2.0421
Epoch 16/50, Train Loss: 1.7904, Val Loss: 2.0604
Epoch 17/50, Train Loss: 1.7139, Val Loss: 1.8939
Epoch 18/50, Tra

[I 2024-09-14 18:18:59,673] Trial 48 finished with value: 0.36986303329467773 and parameters: {'lr': 0.06537749353586599, 'batch_size': 32, 'weight_decay': 4.68773187498937e-05}. Best is trial 39 with value: 0.938356164842844.


Epoch 37/50, Train Loss: 0.3359, Val Loss: 1.3645
Early stopping triggered after 37 epochs
Training complete
Accuracy: 0.6301
F1 Score: 0.5764
Epoch 1/50, Train Loss: 2.8588, Val Loss: 44.1913
Epoch 2/50, Train Loss: 2.4074, Val Loss: 2.5182
Epoch 3/50, Train Loss: 2.3293, Val Loss: 2.4211
Epoch 4/50, Train Loss: 2.2777, Val Loss: 2.2793
Epoch 5/50, Train Loss: 2.2584, Val Loss: 2.2222
Epoch 6/50, Train Loss: 2.2394, Val Loss: 2.1012
Epoch 7/50, Train Loss: 2.2588, Val Loss: 2.2226
Epoch 8/50, Train Loss: 2.2102, Val Loss: 2.3077
Epoch 9/50, Train Loss: 2.1423, Val Loss: 2.1839
Epoch 10/50, Train Loss: 1.9920, Val Loss: 2.0470
Epoch 11/50, Train Loss: 1.9706, Val Loss: 2.2980
Epoch 12/50, Train Loss: 2.0439, Val Loss: 2.2626
Epoch 13/50, Train Loss: 1.8355, Val Loss: 2.0996
Epoch 14/50, Train Loss: 1.7166, Val Loss: 2.3367
Epoch 15/50, Train Loss: 1.6893, Val Loss: 1.9351
Epoch 16/50, Train Loss: 1.5100, Val Loss: 1.6779
Epoch 17/50, Train Loss: 1.2620, Val Loss: 1.7489
Epoch 18/50, Tr

[I 2024-09-14 18:20:28,346] Trial 49 finished with value: 0.11643832921981812 and parameters: {'lr': 0.006329542069221441, 'batch_size': 64, 'weight_decay': 0.00021792124993671002}. Best is trial 39 with value: 0.938356164842844.


Epoch 40/50, Train Loss: 0.0143, Val Loss: 0.9274
Early stopping triggered after 40 epochs
Training complete
Accuracy: 0.8836
F1 Score: 0.8240
Best hyperparameters:  {'lr': 0.012653947233242485, 'batch_size': 64, 'weight_decay': 0.0005945715958985451}
Best value:  0.938356164842844


## Results

```
Best hyperparameters:  {'lr': 0.012653947233242485, 'batch_size': 64, 'weight_decay': 0.0005945715958985451}
Best value:  0.938356164842844
```